# Generates random serial numbers for students. 

* Save the initial gradebook (beginning of term) as BSGrades.csv.
* Also puts in 0 for the grade for the midterm 3

In [ ]:
import pandas as pd

from random import shuffle
df=pd.read_csv('BSGrades.csv')
cols=list(df)
num_students=df.shape[0]
ids=[str(x) for x in range(1000,1000+num_students)]
shuffle(ids)
df['Serial Number Text Grade <Text>']=ids
testname="Midterm 3 Points Grade"
u=[cols.index(x) for x in cols if testname in x][0]
midterm3_column_name = cols[u]
df[midterm3_column_name]=0
df.to_csv('BSGrades.csv', index=False)

# Slaps on coverpage and scratchwork page. Automatically removes the answerkey page

* Make sure the versions with answerkeys are the only ones which end with a digit, e.g. test2_v1.pdf
* Make sure the versions with answerkeys have all small letters in their names, e.g. test2_v1.pdf
* Output version will use same names but all cap letters, e.g TEST2_V1.pdf
* Automatically deduces whether the test is a final or a midterm from the name - make sure finals have the word "final" in the name



In [9]:
from pdfrw import PdfReader, PdfWriter
import glob
import os
from sys import exit


#This code deduces the filenames for the versions with answerkeys. Assumes that these versions end with a digit, e.g. 2019test1v0.pdf etc
pdf_tests=glob.glob("*.pdf")
test_names = [x[:-4] for x in pdf_tests]
tests_ending_with_digits = [x[:-1] for x in test_names if x[-1].isdigit()]
digits = [int(x[-1]) for x in test_names if x[-1].isdigit()]
assert len(set(tests_ending_with_digits))==1, "Multiple tests end with digits. Rename pdf files so that only tests with answerkeys end with digits"
testname = list(set(tests_ending_with_digits))[0]
outputname = testname[:-1].upper()+"_v"


home=os.path.expanduser('~')
joinfilepath=os.path.join(home,'Dropbox','MiscTeachingStuff','pre_n_post')
if not os.path.isdir(joinfilepath):
    print("file not found")
    exit("joinfilepath files not found")

#Checking whether a final or a midterm
fin = 'final' in testname.lower()

    
for n in digits:
    inpfn=testname+str(n)+'.pdf'
    writer = PdfWriter()

    if fin:
        cov_pg=os.path.join(joinfilepath,'FinalCoverPage'+str(n)+'.pdf')
#         op_pg=os.path.join(joinfilepath,'OptionalPage.pdf')
    else:
        cov_pg=os.path.join(joinfilepath,'MidtermCoverPage'+str(n)+'.pdf')
  
    sp=os.path.join(joinfilepath,'scratchwork.pdf')
    
    if os.path.exists(inpfn) and os.path.exists(cov_pg) and os.path.exists(sp):
        writer.addpages(PdfReader(cov_pg).pages)
        writer.addpages(PdfReader(sp).pages)
        writer.addpages(PdfReader(inpfn).pages[:-1])
        opfn=outputname+chr(65+n)+'.pdf'
        print("Writing "+opfn)
        writer.write(opfn)
    else:
        if not os.path.exists(inpfn):
            print(inpfn+' not found')
        if not os.path.exists(mcp):
            print(cov_pg+' not found')
        if not os.path.exists(sp):
            print(sp+' not found')
    
    

Writing 2023S-FINAL_vC.pdf
Writing 2023S-FINAL_vD.pdf


# The following script checks to see if serial numbers are correctly entered in MasteringPhysics.

- Download MP as MP_raw.csv.

- Make sure the Brightspace gradebook is saved as BSGrades.csv

In [ ]:
import pandas as pd
# datafile='MP.csv'
gradebook='BSGrades.csv'
df=pd.read_csv(gradebook, usecols=["Last Name", "First Name", "Serial Number Text Grade <Text>"])
inputfile = 'MP_raw.csv'

df_mp=pd.read_csv(inputfile, usecols=["Last Name", "First Name", "Student ID"], skiprows=3)
df_mp['Student ID'] = df_mp['Student ID'].apply(lambda x: int(x) if x == x else "")
df_mp=df_mp.rename(columns={'Student ID': 'Serial Number Text Grade <Text>'})
df_mp['Last Name']=df_mp['Last Name']+' '+df_mp['First Name']
df_mp.drop('First Name', inplace=True, axis=1)
df_mp=df_mp.rename(columns={'Last Name': 'Name'})
df['Serial Number Text Grade <Text>']=df['Serial Number Text Grade <Text>'].astype(str)
df_mp['Serial Number Text Grade <Text>']=df_mp['Serial Number Text Grade <Text>'].astype(str)
checker_left = pd.merge(df,df_mp[['Serial Number Text Grade <Text>','Name']],on='Serial Number Text Grade <Text>', how='left')
checker_right = pd.merge(df,df_mp[['Serial Number Text Grade <Text>','Name']],on='Serial Number Text Grade <Text>', how='right')
u1=checker_left[checker_left.isnull().any(axis=1)]
u2=checker_left[checker_left['Serial Number Text Grade <Text>'].duplicated(keep=False)]
u3=checker_right[checker_right.isnull().any(axis=1)]

error_rpt=pd.concat([u1, u2, u3], axis=0)
error_rpt["Last Name"] = error_rpt["Last Name"]+' '+error_rpt['First Name']
error_rpt.drop('First Name', axis=1,inplace=True)
error_rpt.rename(columns={'Serial Number Text Grade <Text>': 'Srl No', 'Last Name':'Registered Name', 'Name':'Entered Name'}, inplace=True) 

error_rpt
error_rpt.to_csv("MP_error_rpt.csv")

# The following script prepares the file MP.csv from the MasteringPhysics exported file

# Steps:

- Download masteringphysics  as MP_raw.csv. 

- The following code will create the file MP.csv for use in the next script 
- WARNING: Make sure there are no NaN's (...'s )in the columns of MP_raw (this might happen if a student has been given a deadline extension. Replace the (...) by a 0 before running the script

In [ ]:
import pandas as pd
import numpy as np

inputfile = 'MP_raw.csv'
outputfile = 'MP.csv'
df=pd.read_csv(inputfile, skiprows=3)#reads the file skipping first 3 rows

df = df.replace(np.nan, 0)#replaces original NaN's by 0 so these columns are not removed in th next steps
df = df.replace('--', np.nan)# mark the --'s by Nan's to nuke them
df = df.dropna(axis='columns')
df = df[:-1]# removes the useless last row

# columns_to_drop = ['Login Name', 'Email', 'Enrollment Status', 'Group(s)']
columns_to_drop = ['Enrollment Status', 'Group(s)']
df = df.drop(columns_to_drop, axis=1)

HW_cols = [x for x in list(df) if 'HW' in x]
df['Total Grade'] = df[HW_cols].sum(axis=1)
total = df["Total Grade"][df.index[-1]]
df['MP Score'] = 100*df['Total Grade']/total
df.to_csv(outputfile, index=False)

# The following script transfers MP grades to the (downloaded) BS Gradebook

# Steps:

- Run the previous script to convert MP_raw.csv to MP.csv which is properly formatted for processing. 

- Make sure Brightspace gradebook is downloaded as BSGrades.csv, and "out of" for tests are correct. 

- Set to_print='y' [CLOSE BSGrades.csv if its open]


In [ ]:
import pandas as pd
import os

to_print='y'
df=pd.read_csv('BSGrades.csv')
cols=list(df)

#This part finds the name of the column that has the scores. Assumes that this column has MP in its title
df_mp=pd.read_csv('MP.csv')
cols_mp = list(df_mp)
cols_named_MP=[x for x in cols_mp if "MP" in x]
assert len(cols_named_MP) == 1, "Make sure the Mastering Sccores are in column titled MP Score. No other column should have MP in its header"
MPScores = cols_named_MP[0]

df_mp=df_mp.rename(columns={'Student ID': 'Serial Number Text Grade <Text>'})

df = pd.merge(df,df_mp[['Serial Number Text Grade <Text>',MPScores]],on='Serial Number Text Grade <Text>', how='left')

df['Mastering Physics Points Grade <Numeric MaxPoints:100 Weight:14.75>']=df[MPScores]
df.drop('MP Score', inplace=True, axis=1)
if to_print=='y':
    try:
        df.to_csv('BSGrades.csv', index=False)
        print("Task completed successfully")
    except:
        print('Close BSGrades.csv')

# The following script transfers MP grades and test grades to the (downloaded) BS Gradebook

# Steps:

- Run the previous script to convert MP_raw.csv to MP.csv which is properly formatted for processing. 

- Make sure All_processed.csv is ready (All_processed is the output of Grader)

- Make sure Brightspace gradebook is downloaded as BSGrades.csv, and "out of" for tests are correct. 

- The script will automatically find out which test this is from the folder name, which is something like Test1

- Set to_print='y' [CLOSE BSGrades.csv if its open]

 

The following code will create extra columns in BSGrades for MPScores, test scores and test missed (not done for the final exam). Then it will copy them over to the correct columns and delete the extra columns it created. 

In [ ]:
import pandas as pd
import os

# Trying to deduce which test this is
cwd = os.getcwd()
t = cwd[cwd.rfind(os.sep)-1]
if t in ["1", "2", "3"]:
    which_test = int(t)
else:
    which_test = 4
print("Assuming this is Test {}".format(which_test))
# which_test=1 #Enter 1,2,3 or 4 (for final)    
to_print='y'

df=pd.read_csv('BSGrades.csv')
cols=list(df)

#This part finds the name of the column that has the scores. Assumes that this column has MP in its title
df_mp=pd.read_csv('MP.csv')
cols_mp = list(df_mp)
cols_named_MP=[x for x in cols_mp if "MP" in x]
assert len(cols_named_MP) == 1, "Make sure the Mastering Sccores are in column titled MP Score. No other column should have MP in its header"
MPScores = cols_named_MP[0]

df_mp=df_mp.rename(columns={'Student ID': 'Serial Number Text Grade <Text>'})

df = pd.merge(df,df_mp[['Serial Number Text Grade <Text>',MPScores]],on='Serial Number Text Grade <Text>', how='left')

df_all=pd.read_csv('All_processed.csv')
df_all=df_all.rename(columns={'Srl No': 'Serial Number Text Grade <Text>'})

df = pd.merge(df,df_all[['Serial Number Text Grade <Text>','Score']],on='Serial Number Text Grade <Text>', how='left')
#Copies the scores from All_processed.csv into df under the heading 'Score'

df['Mastering Physics Points Grade <Numeric MaxPoints:100 Weight:14.75>']=df[MPScores]

if which_test<4:
    df = pd.merge(df,df_all[['Serial Number Text Grade <Text>','Missed']],on='Serial Number Text Grade <Text>', how='left')
    #Copies the missed q's from All_processed.csv into df under the heading 'Missed'
    testname="Midterm "+str(which_test)+" Points Grade"#Searches for the entry matching Midterm n
    u=[cols.index(x) for x in cols if testname in x]
    if u:
        df[cols[u[0]]]=df['Score']#copies the column titled Score to that entry
    else:
        print(testname+" not found in BSGrades.csv")
        to_print='n'
    df['Test '+str(which_test)+' Questions Missed Text Grade <Text>']=df['Missed']
    columns_to_drop = ['MP Score', 'Score','Missed']
elif which_test==4:
    testname="Final Points Grade"
    u=[cols.index(x) for x in cols if testname in x]
    if u:
        df[cols[u[0]]]=df['Score']
    else:
        print(testname+" not found in BSGrades.csv")
        to_print='n'
    columns_to_drop = ['MP Score', 'Score']
else:
    print("Bad input for which_test. Should be 1-4")
    to_print='n'

df.drop(columns_to_drop, inplace=True, axis=1)
if to_print=='y':
    try:
        df.to_csv('BSGrades.csv', index=False)
        print("Task completed successfully")
    except:
        print('Close BSGrades.csv')

# The following script calculates letter grades from the numerical grades, and also applies the reward for good dropped midterm. 

* Create the final file as BSGrades_final.csv (this is after uploading the final exam grades and having BS calculate overall grades)
* Adds a column "Adjusted Numerical Grade" which includes the reward for the good dropped midterm. 
* Automatically figures out the available number of points in each midterm.
* Calculates letter grades based on this and stores in a new column "Letter"


In [ ]:
import pandas as pd
import re
def lg(num_grade):
    if num_grade<59.5:
        return 'F'
    elif num_grade>=59.5 and num_grade<62.5:
        return 'D-'
    elif num_grade>=62.5 and num_grade<66.5:
        return 'D'
    elif num_grade>=66.5 and num_grade<69.5:
        return 'D+'
    elif num_grade>=69.5 and num_grade<72.5:
        return 'C-'
    elif num_grade>=72.5 and num_grade<76.5:
        return 'C'
    elif num_grade>=76.5 and num_grade<79.5:
        return 'C+'
    elif num_grade>=79.5 and num_grade<82.5:
        return 'B-'
    elif num_grade>=82.5 and num_grade<86.5:
        return 'B'
    elif num_grade>=86.5 and num_grade<89.5:
        return 'B+'
    elif num_grade>=89.5 and num_grade<92.5:
        return 'A-'
    else:
        return 'A'

def f(x, mta):
    pointmax0=mta[0]
    pointmax1=mta[1]
    pointmax2=mta[2]
    min_midterm = min(x[0]/pointmax0,x[1]/pointmax1,x[2]/pointmax2)
    if min_midterm>=0.7 and min_midterm<0.8:
        return x[3]+.5
    elif min_midterm>=0.8:
        return x[3]+1
    else:
        return x[3]

def striphash(x):
    return x[1:]



df=pd.read_csv('BSGrades_final.csv')
columns = ['Calculated Final Grade Denominator','Adjusted Final Grade Numerator','Adjusted Final Grade Denominator','End-of-Line Indicator']
df.drop(columns, inplace=True, axis=1)
cols=list(df)
testcols = [x for x in cols if "Category:Midterms" in x] # These columns have the test scores
#Make a dictionary which will hold midterm number and total points for that midterm
midterm_totals = {}
for i in range(len(testcols)):
    midterm_no = int(re.findall(r'Midterm\s+\d+|Midterm\d+', testcols[i])[0][-1])
    _ = re.findall(r'MaxPoints:\d+', testcols[i])[0]
    midterm_pts = int(re.findall(r'\d+', _)[0])
    midterm_totals.update( {midterm_no : midterm_pts} )
midterm_totals_array = [midterm_totals[1], midterm_totals[2], midterm_totals[3]]



for n in range(1,4):
    testname="Midterm "+str(n)+" Points Grade"
    testname1="Midterm"+str(n)+" Points Grade" #Just in case the Midterm appears and Midterm1 rather than Midterm 1
    shortname="Midterm "+str(n)
    u=[cols.index(x) for x in cols if testname in x or testname1 in x]
    if len(u)>0:
        cols[u[0]]=shortname
    else:
        print(testname+"not found")
df.columns=cols        
df.rename(columns={'Calculated Final Grade Numerator': 'Numerical Grade'},inplace=True)
df["Adjusted Numerical Grade"] = ""
df["Letter"] = ""

df['Adjusted Numerical Grade'] = df[['Midterm 1','Midterm 2','Midterm 3','Numerical Grade']].apply(lambda x: f(x, midterm_totals_array),  axis=1)
df['Letter'] = df['Adjusted Numerical Grade'].map(lg)
df['OrgDefinedId']=df['OrgDefinedId'].map(striphash)
df.rename(columns={'OrgDefinedId': 'VUnetID'},inplace=True)
try:
    df.to_csv('OverallGrades.csv', index=False)
except:
    print('Close OverallGrades.csv')
    
    
    

#### THE OLD VERSION #######################
# df=pd.read_csv('BSGrades_final.csv')
# columns = ['Calculated Final Grade Denominator','Adjusted Final Grade Numerator','Adjusted Final Grade Denominator','End-of-Line Indicator']
# df.drop(columns, inplace=True, axis=1)
# cols=list(df)
# for n in range(1,4):
#     testname="Midterm "+str(n)+" Points Grade"
#     shortname="Midterm "+str(n)
#     u=[cols.index(x) for x in cols if testname in x]
#     if len(u)>0:
#         cols[u[0]]=shortname
#     else:
#         print(testname+"not found")
# df.columns=cols
# df.rename(columns={'Calculated Final Grade Numerator': 'Numerical Grade'},inplace=True)
# df["Adjusted Numerical Grade"] = ""
# df["Letter"] = ""

# df['Adjusted Numerical Grade'] = df[['Midterm 1','Midterm 2','Midterm 3','Numerical Grade']].apply(f, axis=1)
# df['Letter'] = df['Adjusted Numerical Grade'].map(lg)
# df['OrgDefinedId']=df['OrgDefinedId'].map(striphash)
# df.rename(columns={'OrgDefinedId': 'VUnetID'},inplace=True)
# try:
#     df.to_csv('OverallGrades.csv', index=False)
# except:
#     print('Close OverallGrades.csv')

# The following script transfers final letter grades into the file for uploading to "Online Grading".

* Make sure OverallGrades.csv is ready (previous script)

* Download the file from Online Grading and name it fingra.csv

* Run the script and upload the file finalgrades.csv

In [ ]:
import pandas as pd
df_copyfrom=pd.read_csv('OverallGrades.csv')

df=pd.read_csv('fingra.csv')

df = pd.merge(df,df_copyfrom[['VUnetID','Letter']],on='VUnetID', how='left')
df['Assigned Grade']=df['Letter']
df.drop('Letter', inplace=True, axis=1)
df = df.fillna('')
try:
    df.to_csv('finalgrades.csv', index=False)
except:
    print('Close finalgrades.csv')

# The following script calculates the discussion score and updates BSGrades.

* Download the discussion results (Introductions -> View Topic Statistics) as DiscussionGrades.csv and Tips.csv

* Enter the filenames and appropriate keywords in the dict "discussions", i.e. uncomment the appropriate line "discussions = ..."

* Download the gradebook as BSGrades.csv

* Make sure BSGrades first column is "OrgDefinedId" (will give an assert error if not)

* The script will calculate the scores for each discussion and write them in the correct columns

In [ ]:
import pandas as pd
import glob


def calculate_discussion_score(x): 
    threads=x[1]
    replies=x[2]
    return threads*8+min(2,replies)


discussions = {"DiscussionGrades.csv": "Introduction", "Tips.csv": "Study Tips"}
#discussions = {"DiscussionGrades.csv": "Introduction"}


files = list(discussions.keys())
df=pd.read_csv('BSGrades.csv')
assert 'OrgDefinedId' in list(df), "Make sure BSGrades has OrgDefinedId as a header"
df.rename(columns={'OrgDefinedId': 'Username'},inplace=True)
df['Username']=df['Username'].map(lambda x: x[1:] if x[0]=='#' else x)
for file in files:
    df_disc=pd.read_csv(file)
    list_of_cols=list(df_disc)
    list_to_keep=['[Username]','[Threads]','[Replies]']
    list_to_drop=[x for x in list_of_cols if not x in list_to_keep]
    df_disc.drop(list_to_drop, inplace=True, axis=1)
    df_disc["Score"]=0.0
    df_disc['Score'] = df_disc.apply(calculate_discussion_score, axis=1)
    df_disc.rename(columns={'[Username]': 'Username'},inplace=True)

    cols=list(df)
    n=[cols.index(x) for x in cols if discussions[file] in x][0]#index of the column in BSGrades containing the keyword for that discussion, e.g. "Introductions" 
    copy_to_column=cols[n]

    df = pd.merge(df,df_disc[['Username','Score']],on='Username', how='left')
    df[copy_to_column]=df['Score']
    df.drop('Score', inplace=True, axis=1)
    print("{} score transfer complete".format(file))
df.to_csv('BSGrades.csv', index=False)



# The following script creates a All_fakes.csv files which has student serial numbers, names and fake answers. 

* Make sure BSGrades.csv is ready and Last Name and First Name columns are there

* Automatically checks whether the test is a final from the .tst files. (Final exams have the word "final" in the filename)

* Figures out the number of versions by counting filenames like "test....tst". Make sure alternate tests are named differently, e.g. "Alt_A.tst"



In [ ]:
import random
import glob
import pandas as pd
from grader_funs import getAllKeys, most_frequent

try:
    f = open('BSGrades.csv')
    f.close()
except FileNotFoundError:
    print('Make sure BSGrades.csv is ready!')
    


## The following function creates fake answer data#############
def create_afake(number_of_questions, number_of_versions=3, blanks=False):
    '''creates a fake answerkey'''
    ansstring=[]
    if not blanks:
        choices=[0,1,2,3,4]
    else:
        choices=[0,1,2,3,4,' ']
    for n in range(number_of_questions):
        ans=random.randrange(0,len(choices))
        ansstring.append(str(choices[ans]))
    ans=random.randrange(0,number_of_versions)
    ansstring.append(str(ans))
    ansstring=''.join(ansstring)
    return ansstring
########################################################

# tstfiles=glob.glob("*.tst")
# fin = bool([x for x in tstfiles if "final" in x.lower()])
# if fin:
#     number_of_questions = 25
# else:
#     number_of_questions = 14
        
# number_of_versions=4



# %run grader_funs.py

tstfiles=glob.glob("*.tst")#get all tst files
tstnames=[x[:-5] for x in tstfiles]# make list of the file names
tstname = most_frequent(tstnames)# pick the most common file name, assuming that is the actual test
outs = getAllKeys(tstname)
number_of_questions=outs["numberOfQuestions"]-1
number_of_versions = tstnames.count(tstname)

df=pd.read_csv('BSGrades.csv')
assert( "Last Name" in list(df) and "First Name" in list(df))

df=df[['Serial Number Text Grade <Text>','Last Name','First Name']].copy()

df['Last Name']=df['Last Name']+' '+df['First Name']
df["First Name"]=df["First Name"].apply(lambda x: create_afake(number_of_questions, number_of_versions))
df.to_excel('All_fakes.xlsx', index=False, header=False)

## The following script checks version numbers entered by hand from tests vs version numbers on the scantrons (to make sure students did the test they were assigned).

* Enter the version numbers by hand into All.xlsx in the column next to the answers.

* Run the script below

In [ ]:
import pandas as pd
xls_file = pd.ExcelFile('All.xlsx')
df = xls_file.parse('Sheet1', header=None, parse_cols=3,names = ["Srl No", "Name", "Answers",'Version'], dtype='str')
df.loc[df['Answers'].str[-1]!=df['Version']]

## The following script takes a file of manual entries (ABBCDE.....) and converts them into a string of numbers (011234....) and prepares All.xlsx

* Make sure the input file All_manual.xlsx has first column as Serial number, second columns as name and third column as manual entries (best to start with All_fakes.xlsx
* Spaces between letters are ok. Mark blanks by the letter S
* Run the script. A file named All.xlsx should be created




In [ ]:
# Setting up processing of manual entries
def numerify_manual(ans):
    ans = ''.join(ans.split())
    numerified = ''.join([str(ord(x)-65) for x in ans])
    rid_S = numerified.replace('18', ' ') #'18' is what 'S" converts to (blanks) which are being replaced by spaces
    return rid_S

import pandas as pd
datafilename = 'All_manual.xlsx'
df = pd.read_excel(datafilename, header=None, usecols=[0,1,2], names = ["Srl No", "Name", "Answers"], dtype='str')
df["Numerified"] = df['Answers'].map(numerify_manual)
df = df[['Srl No', 'Name', 'Numerified','Answers']]
df.to_excel('All.xlsx', header = False, index = False)

## The following script takes two lists of manual entries [Adrianna Kow, Babcock Ghosh...], [ABBCDE....., BCCDE....., ...]) and converts them into a string of numbers (011234....) and adds them on to an existing All.xlsx

* Make sure the All.xlsx is ready
* Spaces between letters are ok. Mark blanks by the letter S
* Run the script. All.xlsx should be have the new entries entered in the end

In [ ]:
def numerify_manual(ans):
    ans = ''.join(ans.split())
    numerified = ''.join([str(ord(x)-65) for x in ans])
    rid_S = numerified.replace('18', ' ') #'18' is what 'S" converts to (blanks) which are being replaced by spaces
    return rid_S


import pandas as pd
datafilename = 'All.xlsx'
out_datafilename = "All.xlsx"
df = pd.read_excel(datafilename, header=None, usecols=[0,1,2], names = ["Srl No", "Name", "Answers"], dtype='str')

# Enter serial number in text
srl_list = [1041,1144, 1124, 1138 ]
srl_list=[str(x) for x in srl_list]
name_list = ["Daniel Shade","Harrison Rogers", "Michael Bruno", "Oliver Jimenez"]
anslist = ['DEBAB CCBDD BEABA EBEA',
           'DEAEB ABBDB AECBD EDCA',
           'DEDAB CBACD BEAED EDEA',
           'BEDAB BCCAD DBAED EEEA'
            
          ]



assert len(name_list)==len(anslist) == len(srl_list), "srl_list, name_list and anslist much have the same length"
name_list = [x.upper() for x in name_list]
numerical_answers = [numerify_manual(x) for x in anslist]
data_tuples = list(zip(srl_list,name_list, numerical_answers))
df_additions = pd.DataFrame(data_tuples, columns=list(df))
df_added=df.append(df_additions,ignore_index=True)
df_added['Answers']=df_added['Answers'].apply(str)

# try:
#     myfile = open(datafilename, "r+") # or "a+", whatever you need
# except IOError:
#     print("Could not open file! Please close Excel!")
# df_added.to_excel(datafilename)
df_added.to_excel(out_datafilename, header = False, index = False)



# DEVELOPMENT AREA
## All code below is in development

In [ ]:
import pandas as pd
import os

# Trying to deduce which test this is
# cwd = os.getcwd()
# t = cwd[cwd.rfind(os.sep)-1]
# if t in ["1", "2", "3"]:
#     which_test = int(t)
# else:
#     which_test = 4
# print("Assuming this is Test {}".format(which_test))
which_test=3 #Enter 1,2,3 or 4 (for final)    
to_print='n'

df=pd.read_csv('BSGrades.csv')
cols=list(df)

#This part finds the name of the column that has the scores. Assumes that this column has MP in its title
df_mp=pd.read_csv('MP.csv')
cols_mp = list(df_mp)
cols_named_MP=[x for x in cols_mp if "MP" in x]
assert len(cols_named_MP) == 1, "Make sure the Mastering Sccores are in column titled MP Score. No other column should have MP in its header"
MPScores = cols_named_MP[0]

df_mp=df_mp.rename(columns={'Student ID': 'Serial Number Text Grade <Text>'})

df = pd.merge(df,df_mp[['Serial Number Text Grade <Text>',MPScores]],on='Serial Number Text Grade <Text>', how='left')

df['Mastering Physics Points Grade <Numeric MaxPoints:100 Weight:14.75>']=df[MPScores]
df.drop('MP Score', inplace=True, axis=1)
if to_print=='y':
    try:
        df.to_csv('BSGrades.csv', index=False)
        print("Task completed successfully")
    except:
        print('Close BSGrades.csv')

In [ ]:
midterm_totals_array

In [ ]:
MPScores

In [ ]:
df_mp.head()

In [ ]:
tstfiles

In [ ]:

print(tests_ending_with_digits)

In [1]:
2+2

4

In [ ]:
import pandas 